<a href="https://colab.research.google.com/github/varun1414/youtube_class/blob/master/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from fastai import *

In [4]:
import pandas as pd


In [14]:
df=pd.read_csv('./sample_data/Youtube Video Dataset.csv')
df

,Title,Videourl,Category,Description
0,Madagascar Street Food!!! Super RARE Malagasy ...,/watch?v=EwBA1fOQ96c,Food,🎥GIANT ALIEN SNAIL IN JAPAN! » https://youtu.b...
1,42 Foods You Need To Eat Before You Die,/watch?v=0SPwwpruGIA,Food,This is the ultimate must-try food bucket list...
2,Gordon Ramsay’s Top 5 Indian Dishes,/watch?v=upfu5nQB2ks,Food,We found 5 of the best and most interesting In...
3,How To Use Chopsticks - In About A Minute 🍜,/watch?v=xFRzzSF_6gk,Food,You're most likely sitting in a restaurant wit...
4,Trying Indian Food 1st Time!,/watch?v=K79bXtaRwcM,Food,HELP SUPPORT SINSTV!! Shop Our Sponsors!\nLast...
...,...,...,...,...
11206,"art journal | shimmer sprays, stencils, collag...",/watch?v=mJBOQr_-8_Y,Art&Music,Step by step video on creating an art journal ...
11207,Ar-Tea Collage * Mixed Media Art,/watch?v=GOihUhnZxtw,Art&Music,"By: Ilene McInnes,\nMixed media Art and inspir..."
11208,DIY Mixed Media Art Collage Greeting Cards / M...,/watch?v=BchUtLfjews,Art&Music,Make your own Mixed Media Greeting Cards\n\nHe...
11209,Art Collage Process DecoJournal using Rice Pap...,/watch?v=FlQ_j-cflQc,Art&Music,Art Collage Process DecoJournal using Rice Pap...


In [17]:
df['Category'].unique()

array(['Food', 'manufacturing', 'History', 'travel blog',
       'Science&Technology', 'Art&Music'], dtype=object)

In [ ]:
df['labels']=df.Category.apply(lambda x: if )

      7  0  0.1  0.2  0.3  0.4  ...  0.662  0.663  0.664  0.665  0.666  0.667
0     2  0    0    0    0    0  ...      0      0      0      0      0      0
1     1  0    0    0    0    0  ...      0      0      0      0      0      0
2     0  0    0    0    0    0  ...      0      0      0      0      0      0
3     4  0    0    0    0    0  ...      0      0      0      0      0      0
4     1  0    0    0    0    0  ...      0      0      0      0      0      0
...  .. ..  ...  ...  ...  ...  ...    ...    ...    ...    ...    ...    ...
9994  2  0    0    0    0    0  ...      0      0      0      0      0      0
9995  3  0    0    0    0    0  ...      0      0      0      0      0      0
9996  4  0    0    0    0    0  ...      0      0      0      0      0      0
9997  5  0    0    0    0    0  ...      0      0      0      0      0      0
9998  6  0    0    0    0    0  ...      0      0      0      0      0      0

[9999 rows x 785 columns]


# New Section

In [ ]:
tr='https://amzn.to/2r67r1z hello hi https://youtu.be/5jcgwu-gvm4'
stsr='#hi yo\
      #kok #bye team.befrs@gmail.com'
pat=r'[a-zA-Z0-9+_.]+[a-z0-9]+@[a-z0-9A-Z]+\.[com]*[net]*'
cat=r'#[\w]+'
sat=r'[http|ftp|https:]+\/\/+[\w\-_]+\.[\w\-_]+[\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#]+'
m=re.findall(sat,tr)
tr=tr.split()
tr
# pat=r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+"
# 
temp=[e for e in tr if e in m ]
print(m)

['https://amzn.to/2r67r1z', 'https://youtu.be/5jcgwu-gvm4']
